# HTMX Integration

> HTMX patterns and helpers for daisyUI components in FastHTML

In [ ]:
#| default_exp core.htmx

In [ ]:
#| export
from typing import Dict, Any, Optional, List, Union, Literal, Callable
from dataclasses import dataclass, field
from enum import Enum
from fasthtml.common import *
from cjm_fasthtml_daisyui.core.types import (
    # Type aliases
    HTMLAttrs,
    CSSClasses,
    HTMXValue,
    # Enums
    HTMXTrigger,
    HTMXSwap,
    SemanticColor,
    DaisyComponentType,
    # Utility functions
    ensure_dict
)
from cjm_fasthtml_daisyui.core.base import ValidatedDaisyComponent, DaisyComponent

## HTMX Integration Patterns

This module provides HTMX-aware patterns for daisyUI components, embracing the FastHTML philosophy of server-side rendering with progressive enhancement.

In [ ]:
#| export
@dataclass
class HTMXAttrs:
    """Container for HTMX attributes with type-safe values."""
    hx_get: Optional[str] = None
    hx_post: Optional[str] = None
    hx_put: Optional[str] = None
    hx_patch: Optional[str] = None
    hx_delete: Optional[str] = None
    hx_trigger: Optional[str] = None
    hx_target: Optional[str] = None
    hx_swap: Optional[str] = None
    hx_indicator: Optional[str] = None
    hx_push_url: Optional[Union[bool, str]] = None
    hx_select: Optional[str] = None
    hx_select_oob: Optional[str] = None
    hx_vals: Optional[HTMXValue] = None  # Using HTMXValue type alias
    hx_confirm: Optional[str] = None
    hx_disable: Optional[bool] = None
    hx_disabled_elt: Optional[str] = None
    hx_include: Optional[str] = None
    hx_ext: Optional[str] = None
    
    def to_dict(
        self
    ) -> HTMLAttrs:  # Dictionary of HTML attributes with HTMX prefixes
        """Convert to dictionary of HTML attributes."""
        attrs: HTMLAttrs = {}
        for key, value in self.__dict__.items():
            if value is not None:
                # Convert Python snake_case to HTMX hyphenated attributes
                attr_name = key.replace('_', '-')
                
                # Handle boolean attributes
                if isinstance(value, bool):
                    attrs[attr_name] = str(value).lower()
                # Handle dictionary values (for hx-vals)
                elif isinstance(value, dict):
                    import json
                    attrs[attr_name] = json.dumps(value)
                else:
                    attrs[attr_name] = value
        
        return attrs

## HTMX-Aware Base Component

Extending DaisyComponent with HTMX support:

In [ ]:
#| export
class HTMXComponent(ValidatedDaisyComponent):
    """Base class for HTMX-aware daisyUI components.
    
    Extends ValidatedDaisyComponent with HTMX attributes and helper methods
    for building interactive components following FastHTML patterns.
    """
    
    def __init__(self, *args, htmx: Optional[HTMXAttrs] = None, **kwargs):
        """Initialize with optional HTMX attributes."""
        super().__init__(*args, **kwargs)
        self.htmx = htmx or HTMXAttrs()
    
    def with_htmx(
        self,
        get: Optional[str] = None,  # URL endpoint for GET request
        post: Optional[str] = None,  # URL endpoint for POST request
        put: Optional[str] = None,  # URL endpoint for PUT request
        patch: Optional[str] = None,  # URL endpoint for PATCH request
        delete: Optional[str] = None,  # URL endpoint for DELETE request
        trigger: Optional[Union[HTMXTrigger, str]] = None,
        target: Optional[str] = None,  # CSS selector for element to update
        swap: Optional[Union[HTMXSwap, str]] = None,
        **kwargs
    ) -> 'HTMXComponent':  # Self for method chaining
        """Configure HTMX attributes fluently."""
        if get:
            self.htmx.hx_get = get
        if post:
            self.htmx.hx_post = post
        if put:
            self.htmx.hx_put = put
        if patch:
            self.htmx.hx_patch = patch
        if delete:
            self.htmx.hx_delete = delete
        
        if trigger:
            self.htmx.hx_trigger = trigger.value if isinstance(trigger, HTMXTrigger) else trigger
        if target:
            self.htmx.hx_target = target
        if swap:
            self.htmx.hx_swap = swap.value if isinstance(swap, HTMXSwap) else swap
        
        # Handle additional kwargs
        for key, value in kwargs.items():
            if hasattr(self.htmx, f"hx_{key}"):
                setattr(self.htmx, f"hx_{key}", value)
        
        return self
    
    def with_loading(
        self,
        indicator_id: str,  # ID of the loading indicator element to show during request
        disable_during: Optional[str] = None  # CSS selector of elements to disable during request
    ) -> 'HTMXComponent':  # Self for method chaining
        """Configure loading indicators."""
        self.htmx.hx_indicator = f"#{indicator_id}"
        if disable_during:
            self.htmx.hx_disabled_elt = disable_during
        return self
    
    def with_confirmation(
        self,
        message: str  # Confirmation message to display before request
    ) -> 'HTMXComponent':  # Self for method chaining
        """Add confirmation dialog."""
        self.htmx.hx_confirm = message
        return self
    
    def render_attrs(
        self
    ) -> HTMLAttrs:  # Combined dictionary of all HTML and HTMX attributes
        """Build all HTML attributes including HTMX."""
        attrs = super().render_attrs()
        
        # Add HTMX attributes
        htmx_attrs = self.htmx.to_dict()
        attrs.update(htmx_attrs)
        
        return attrs

## Helper Functions

Utility functions for HTMX integration:

In [ ]:
#| export
def htmx_attrs(
    **kwargs
) -> HTMLAttrs:  # Dictionary with proper HTMX attribute names
    """Convert keyword arguments to HTMX attributes."""
    attrs: HTMLAttrs = {}
    for key, value in kwargs.items():
        # Convert to hx- prefix
        if not key.startswith('hx_'):
            key = f'hx_{key}'
        
        # Convert underscores to hyphens
        attr_name = key.replace('_', '-')
        
        # Handle special conversions
        if isinstance(value, HTMXTrigger):
            value = value.value
        elif isinstance(value, HTMXSwap):
            value = value.value
        elif isinstance(value, bool):
            value = str(value).lower()
        elif isinstance(value, dict):
            import json
            value = json.dumps(value)
        
        attrs[attr_name] = value
    
    return attrs

In [ ]:
#| export
def loading_indicator(
    indicator_id: str,  # ID for the indicator element
    text: str = "Loading...",  # Text to display next to spinner
    size: str = "md"  # Size of spinner (xs, sm, md, lg, xl)
) -> FT:  # Loading indicator element
    """Create a loading indicator element.
    
    Creates a daisyUI loading spinner with text, initially hidden.
    Use with HTMXComponent.with_loading() to show during requests.
    """
    return Div(
        Span(cls=f"loading loading-spinner loading-{size}"),
        " ",
        text,
        id=indicator_id,
        cls="htmx-indicator"
    )

In [ ]:
#| export
def oob_alert(
    message: str,  # Alert message to display
    alert_type: str = "info",  # Type of alert (info, success, warning, error)
    target_id: str = "alerts",  # ID of container element to append alert to
    auto_dismiss: Optional[int] = 5000  # Auto dismiss after milliseconds (None to disable)
) -> FT:  # Alert element with out-of-band swap attribute
    """Create out-of-band alert message.
    
    Creates a daisyUI alert that can be swapped out-of-band into any page.
    Useful for showing feedback messages after HTMX requests.
    """
    alert_id = f"alert-{id(message)}"
    
    alert = Div(
        message,
        id=alert_id,
        cls=f"alert alert-{alert_type}",
        hx_swap_oob=f"beforeend:#{target_id}"
    )
    
    if auto_dismiss:
        alert.attrs[f"hx-trigger"] = f"load delay:{auto_dismiss}ms"
        alert.attrs["hx-swap"] = "outerHTML"
        alert.attrs["hx-target"] = f"#{alert_id}"
        alert.attrs["hx-get"] = "/empty"  # Endpoint that returns empty response
    
    return alert

## Usage Examples

Examples of using HTMX patterns with daisyUI components:

### Basic HTMX Component

In [ ]:
# Example: Using component type enum with HTMX
# Show how ValidatedDaisyComponent provides type safety
print("Component type validation example:")
print(f"Button enum value: {DaisyComponentType.BUTTON.value}")
print(f"Button category: {DaisyComponentType.BUTTON.get_category()}")

# You can also validate component types
all_action_components = DaisyComponentType.get_actions()
print(f"\nAll action components: {[c.value for c in all_action_components]}")

Component type validation example:
Button enum value: btn
Button category: actions

All action components: ['btn', 'dropdown', 'modal', 'swap', 'theme-controller']


In [ ]:
# Example: Create an HTMX-aware button using ValidatedDaisyComponent
class HTMXButton(HTMXComponent):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.component_type = DaisyComponentType.BUTTON

# Create button with HTMX behavior
button = HTMXButton()
button.with_htmx(
    get="/api/data",
    trigger=HTMXTrigger.CLICK,
    target="#results",
    swap=HTMXSwap.INNER_HTML
).with_loading("loading-spinner")

# Get the attributes
attrs = button.render_attrs()
print("Button attributes:")
for key, value in sorted(attrs.items()):
    print(f"  {key}: {value}")
    
print(f"\nComponent type: {button.component_type}")
print(f"Component class: {button.component_class()}")

Button attributes:
  class: btn
  hx-get: /api/data
  hx-indicator: #loading-spinner
  hx-swap: innerHTML
  hx-target: #results
  hx-trigger: click

Component type: DaisyComponentType.BUTTON
Component class: btn


## Best Practices

1. **Server-Side First**: Embrace FastHTML's server-side rendering philosophy
2. **Progressive Enhancement**: Components work without JavaScript, HTMX enhances them
3. **Semantic HTML**: Use proper HTML elements (dialog, details, form)
4. **Loading States**: Always provide visual feedback during requests
5. **Error Handling**: Use OOB swaps for error messages
6. **Accessibility**: Ensure HTMX enhancements maintain accessibility

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()